## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
data = pd.read_csv('/datasets/data.csv')
data.head(50)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

таблица состоит из 12 столбцов. Так же в представленной таблице увидел, что в столбце 'days_employed' есть отрицательные значения и слишком большие, в столбце 'purpose' одни и те же категории названы по разному, так же как и в столбце 'education' и 'purpose'

In [2]:
#рассмотрим типы данных и общую информацию
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

столбцы 'days_employed' и 'total_income' имеет тип float64, считаю что стаж и доход должны быть целыми числоми.
так же в этих столбцах есть пропущенные значения

In [3]:
# получим некоторые основные статистические данные таблица data
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Вывод

В столбце 'children' есть отрицательное значение -1, а максимальное значение по 'children' - 20 (это возможно, но подозрительно). Столбец 'days_employed' - 173 года (63046 дней) - явно не корректное значение.
В столбце 'dob_years' минимальное значение равено 0

### В каждом из столбцов найду уникальные значения

In [4]:
# дети
data['children'].value_counts().sort_index()

-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

In [5]:
# трудовой стаж (поиск отрицательных значений)
data[data['days_employed'] < 0]['days_employed'].count()

15906

In [6]:
# трудовой стаж (слишком большие значения)
data[data['days_employed'] > 0].sort_values(['days_employed']).head(50)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
20444,0,328728.720605,72,среднее,1,вдовец / вдова,2,F,пенсионер,0,96519.339647,покупка жилья для семьи
9328,2,328734.923996,41,высшее,0,женат / замужем,0,M,пенсионер,0,126997.497760,операции со своей недвижимостью
17782,0,328771.341387,56,среднее,1,женат / замужем,0,F,пенсионер,0,68648.047062,операции с коммерческой недвижимостью
14783,0,328795.726728,62,высшее,0,женат / замужем,0,F,пенсионер,0,79940.196752,на покупку своего автомобиля
7229,1,328827.345667,32,среднее,1,гражданский брак,1,F,пенсионер,0,122162.965695,сыграть свадьбу
5332,0,328834.274455,55,среднее,1,женат / замужем,0,F,пенсионер,0,49802.142892,профильное образование
5756,0,328842.364916,51,высшее,0,женат / замужем,0,F,пенсионер,0,189181.456572,недвижимость
13325,0,328862.571412,55,среднее,1,вдовец / вдова,2,F,пенсионер,0,99127.379599,на покупку автомобиля
3642,0,328874.270668,54,среднее,1,женат / замужем,0,F,пенсионер,0,202345.082322,строительство собственной недвижимости
3386,0,328895.862752,58,Среднее,1,вдовец / вдова,2,F,пенсионер,0,130432.389679,образование


In [7]:
# возраст
data['dob_years'].value_counts().sort_index()

0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [8]:
# уровень образования
data['education'].value_counts().sort_index()

ВЫСШЕЕ                   274
Высшее                   268
НАЧАЛЬНОЕ                 17
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
Начальное                 15
Неоконченное высшее       47
СРЕДНЕЕ                  772
Среднее                  711
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
высшее                  4718
начальное                250
неоконченное высшее      668
среднее                13750
ученая степень             4
Name: education, dtype: int64

In [9]:
# семейное положение
data['family_status'].value_counts().sort_index()

Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
гражданский брак          4177
женат / замужем          12380
Name: family_status, dtype: int64

In [10]:
# пол
data['gender'].value_counts().sort_index()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [11]:
# тип занятости
data['income_type'].value_counts().sort_index()

безработный            2
в декрете              1
госслужащий         1459
компаньон           5085
пенсионер           3856
предприниматель        2
сотрудник          11119
студент                1
Name: income_type, dtype: int64

In [12]:
# цель получения
data['purpose'].value_counts().sort_values(ascending=False)

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

### Итоговый вывод

Таблица состоит 12 столбцов, в ней 21 525 записей. 
Столбец 'children' имеет 47 значений -1, выглядит как ошибка выгрузки (исправим знаение на 0) и 76 значений, где у клиента 20 детей (явно ошибка, избавимся от этих значений, так жеисправив на 0).
Столбец 'days_employed' имеет 15906 отрицательных значений, будем рассматривать значения по модулю.
В столбце 'education' есть одни и те же категории в разном регистре – сделаем всё одного вида и избавимся от дубликатов.
В столбце 'purpose' та же картина, думаю целесообразно будет применить лемматизацию


### Шаг 2. Предобработка данных

### Обработка пропусков

Заполним пропуски в столбце 'days_employed'. Для этого добавим новый столбец ratio_days_employed, чтобы впоследствии заполнить NaN в столбце 'days_employed' с учетом среднего этого показателя по группе. в расчётах учичтывал, что трудоустроиться возможно по достижении 18 лет

In [13]:
# избавимся от отрицательных значений и заменим значения NaN в датафрейме 
data[['total_income', 'days_employed']] = data[['total_income', 'days_employed']].abs()
data['days_employed'] = data['days_employed'].fillna(value = data['days_employed'].median())
data['total_income'] = data['total_income'].fillna(value = data['total_income'].median())
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [14]:
# удалим неверные значения в столбце 'children' 
data = data[(data['children'] >= 0) & (data['children'] <= 5)]
data.sort_values('children')['children'].value_counts().sort_index()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

In [15]:
# проверим остались ли пропуски
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
children            21402 non-null int64
days_employed       21402 non-null float64
dob_years           21402 non-null int64
education           21402 non-null object
education_id        21402 non-null int64
family_status       21402 non-null object
family_status_id    21402 non-null int64
gender              21402 non-null object
income_type         21402 non-null object
debt                21402 non-null int64
total_income        21402 non-null float64
purpose             21402 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


### Вывод

В результате видим, что пропусков не осталось. Т.е. имеем только те данные, которые необходимы для последующего анализа



### Замена типа данных

In [16]:
# в столбцах 'days_empoyed' и 'total_income' заменим тип данных на integer
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
# проверим методом info
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
children            21402 non-null int64
days_employed       21402 non-null int64
dob_years           21402 non-null int64
education           21402 non-null object
education_id        21402 non-null int64
family_status       21402 non-null object
family_status_id    21402 non-null int64
gender              21402 non-null object
income_type         21402 non-null object
debt                21402 non-null int64
total_income        21402 non-null int64
purpose             21402 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Вывод

Использовал метод astype(), т.к. to_numeric служит для конвертации строк и приводит их к ввещественным величинам с плавающей точкой.



### Обработка дубликатов

#### чтобы перейти к поиску дубликатов необходимо привести таблицу в читаемый вид, а именно уровень образования клиента к  единому регистру


In [17]:
data['education'] = data['education'].str.lower()

In [18]:
data.head(50)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


In [19]:
# узнаем сколько дубликатов имеется в датайрейме
data.duplicated().sum()

71

In [20]:
# рассмотрим какие именно это за дубликакты
data[data.duplicated(keep=False)].sort_values(by=['total_income', 'days_employed'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
120,0,2194,46,среднее,1,женат / замужем,0,F,сотрудник,0,145017,высшее образование
520,0,2194,35,среднее,1,гражданский брак,1,F,сотрудник,0,145017,сыграть свадьбу
541,0,2194,57,среднее,1,женат / замужем,0,F,сотрудник,0,145017,сделка с подержанным автомобилем
554,0,2194,60,среднее,1,женат / замужем,0,M,сотрудник,0,145017,покупка недвижимости
680,1,2194,30,высшее,0,женат / замужем,0,F,госслужащий,0,145017,покупка жилья для семьи
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,2194,64,среднее,1,женат / замужем,0,F,пенсионер,0,145017,дополнительное образование
21032,0,2194,60,среднее,1,женат / замужем,0,F,пенсионер,0,145017,заняться образованием
21132,0,2194,47,среднее,1,женат / замужем,0,F,сотрудник,0,145017,ремонт жилью
21281,1,2194,30,высшее,0,женат / замужем,0,F,сотрудник,0,145017,покупка коммерческой недвижимости


### Вывод

В таблице видно, что это и не дубликаты вовсе, т.к. совпадает всего один столбец 'total_income'. Следовательнно делаю вывод, что ничего не повлияет на правильность конечного результата, по крайней мере на этом этапе.



### Лемматизация

В столбце 'purpose' хранятся цели получения кредита, многие из них одинаковые,  но описаны по разному. Вычислим количествоо уникальных лемм.

In [174]:
# ещё раз посмотрим список
data['purpose'].value_counts().sort_values(ascending=False)

свадьба                                   796
на проведение свадьбы                     772
сыграть свадьбу                           769
операции с недвижимостью                  673
покупка коммерческой недвижимости         661
покупка жилья для сдачи                   651
операции с жильем                         648
операции с коммерческой недвижимостью     646
жилье                                     642
покупка жилья                             641
покупка жилья для семьи                   640
недвижимость                              632
строительство собственной недвижимости    628
операции со своей недвижимостью           626
строительство жилой недвижимости          622
строительство недвижимости                620
покупка недвижимости                      619
покупка своего жилья                      619
ремонт жилью                              609
покупка жилой недвижимости                603
на покупку своего автомобиля              504
заняться высшим образованием      

In [175]:
# найдём самые частоиспользуемые слова, сразу откинув пробелы и предлоги 
from collections import Counter
m = Mystem()
purposes = []

for word in data['purpose']:
    lemma = m.lemmatize(word)
    purposes += filter(lambda l: l not in [' ', '\n', 'с', 'со', 'на'], lemma)

print(Counter(purposes))

Counter({'недвижимость': 6330, 'покупка': 5880, 'жилье': 4450, 'автомобиль': 4288, 'образование': 3997, 'операция': 2593, 'свадьба': 2337, 'свой': 2224, 'строительство': 1870, 'высокий': 1368, 'получение': 1311, 'коммерческий': 1307, 'для': 1291, 'жилой': 1225, 'сделка': 939, 'заниматься': 908, 'дополнительный': 902, 'проведение': 772, 'сыграть': 769, 'сдача': 651, 'семья': 640, 'собственный': 628, 'ремонт': 609, 'подержанный': 484, 'подержать': 472, 'приобретение': 460, 'профильный': 432})


In [176]:
# в итоговм списке целей оставим только наибболее частовстречаемые категории, ремонт не самая частая, но всё же уникальна
purpose_category = [
    'недвижимость',
    'жилье',
    'автомобиль',
    'образование',
    'свадьба'
]

### Вывод

По результатам лемматизации выявил 6 категорий по которым буду проводить категоризацию, с целью дальнейшего анализа и составления выводов


### Категоризация данных

In [177]:
def category_purpose(purpose):
    lemmas = m.lemmatize(purpose)
    for category in purpose_category:
        for lemma in lemmas:
            if category in lemma:
                return category

In [178]:
# к нашему датафрейму добавим новый столбецс с категориями, использовав метод apply
data['purpose_category'] = data['purpose'].apply(category_purpose)
# проверим добавился ли столбец в татблицу и корректно ли он работает
data.head(50)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,жилье
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,жилье
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,жилье


In [179]:
# далее разделим заёмщиков на категории по количеству детей
def category_children(children):
    if children == 0:
        return 'нет детей'
    if children <=2:
        return 'среднестатистические'
    if children >= 3:
        return 'многодетные'

In [180]:
# добавим ещё один столбец, с категориями по количеству детей столбец 
data['children_category'] = data['children'].apply(category_children)
data.head(50)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,children_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,среднестатистические
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,среднестатистические
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,нет детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,многодетные
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,нет детей
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,жилье,нет детей
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,жилье,нет детей
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,нет детей
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,среднестатистические
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,жилье,нет детей


добавим столбец, с категориями по семейному положению, т.е. сократим количество категорий, для более удобного анализа

In [181]:
def category_family_status(family_status):
    if family_status == 'женат / замужем':
        return 'в браке'
    if family_status == 'гражданский брак':
        return 'в браке'
    
    return 'не в браке'

In [182]:
data['family_status_category'] = data['family_status'].apply(category_family_status)
data.head(50)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,children_category,family_status_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,среднестатистические,в браке
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,среднестатистические,в браке
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,нет детей,в браке
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,многодетные,в браке
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,нет детей,в браке
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,жилье,нет детей,в браке
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,жилье,нет детей,в браке
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,нет детей,в браке
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,среднестатистические,в браке
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,жилье,нет детей,в браке


In [183]:
# проведём категоризацию по уровню дохода
def category_total_income(total_income):
    if total_income < 50000:
        return 'низкий'
    elif total_income <= 350000:
        return 'средний'
    elif total_income < 350000:
        return 'выше среднего'
    else:
        return 'высокий'

In [184]:
# добавим этот столбец в датафрейм
data['total_income_category'] = data['total_income'].apply(category_total_income)
data.head(50)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,children_category,family_status_category,total_income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,среднестатистические,в браке,средний
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,среднестатистические,в браке,средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,нет детей,в браке,средний
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,многодетные,в браке,средний
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,нет детей,в браке,средний
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,жилье,нет детей,в браке,средний
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,жилье,нет детей,в браке,средний
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,нет детей,в браке,средний
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,среднестатистические,в браке,средний
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,жилье,нет детей,в браке,средний


### Вывод

Таблица готова к анализу, категоризация произведена. Можно приступать к ответам на вопросы

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [185]:
# составим сводную таблицу
relation_children = data.groupby('children_category')['debt'].agg(['mean', 'sum'])
format_dict = { 'mean': '{:.2%}' }
relation_children.style.format(format_dict)

,mean,sum
children_category,,
многодетные,8.16%,31
нет детей,7.51%,1063
среднестатистические,9.28%,638


In [186]:
data['children_category'].value_counts()

нет детей               14149
среднестатистические     6873
многодетные               380
Name: children_category, dtype: int64

### Вывод

Бездетные чаще выплачивают кредит в срок, считаю что это не удивительно, гораздо проще планировать бюджет, когда нет детей,так же видно что это самая большая категория. В среднестатистических семьях, где 1-2 детей, отстают почти на 2 процента, хотя обращений в банк от этой категории в 2 раза ниже. Ну и в заключении многодетные семьи, их достаточно мало, но тем не менее ситуация по возвратам всё равно хуже, нежели у бездетных. Следовательно можно сделать вывод, что основными клиентами банка являются люди без детей и же лучше всего справляются с выплатами кредита в срок.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [187]:
relation_family_status = data.groupby('family_status_category')['debt'].agg(['mean', 'sum'])
format_dict = { 'mean': '{:.2%}' }
relation_family_status.style.format(format_dict)

,mean,sum
family_status_category,,
в браке,7.97%,1312
не в браке,8.50%,420


In [188]:
data['family_status_category'].value_counts()

в браке       16462
не в браке     4940
Name: family_status_category, dtype: int64

### Вывод

Клиенты находящщиеся в браке чаще выплачивают кредит в срок и в то же время примерно в 3 раза чаще обращаются для получения кредита. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [189]:
relation_total_income = data.groupby('total_income_category')['debt'].agg(['mean', 'sum'])
format_dict = { 'mean': '{:.2%}' }
relation_total_income.style.format(format_dict)

,mean,sum
total_income_category,,
высокий,6.43%,55
низкий,6.20%,23
средний,8.20%,1654


In [190]:
data['total_income_category'].value_counts()

средний    20175
высокий      856
низкий       371
Name: total_income_category, dtype: int64

### Вывод

Чащё всего за кредитом обращаются люди со средним заработком и в то же время они чаще не могут выплатить его в срок. Интересным для меня оказалось, что люди с низким уровнем дохода реже всего уходят в 'просрочку', видимо небольшой доход заставляет бережнее относиться к деньгам. Люди с высоким уровнем дохода почти не отстают от людей с низким, видимо так же осознают ценность денег.

- Как разные цели кредита влияют на его возврат в срок?

In [191]:
relation_purpose = data.groupby('purpose_category')['debt'].agg(['mean', 'sum'])
format_dict = { 'mean': '{:.2%}' }
relation_purpose.style.format(format_dict)

,mean,sum
purpose_category,,
автомобиль,9.33%,400
жилье,6.92%,308
недвижимость,7.46%,472
образование,9.23%,369
свадьба,7.83%,183


In [192]:
data['purpose_category'].value_counts()

недвижимость    6330
жилье           4450
автомобиль      4288
образование     3997
свадьба         2337
Name: purpose_category, dtype: int64

### Вывод

Как видно в таблице, те кто хотят купить жильё или др недвижимость более отвественно относятся к выплатам, они же ии чаще других обращаются за кредитом. Самая плохая картина у тех, кто хочет автомобиль и учиться, эти категория не сильно отстаёт по количеству заёмщиков от тех, кто покупает квартиру



### Шаг 4. Общий вывод

Подводя итог могу сказать, что самые ответственные заёмщики имеют низкий или высокий заработок, находятся в браке и не имеют детей, их цель купить квартиру.

